# Парсер HeadHunter
- ссылка вакансии
- название вакансии
- название компании
- страна
- город
- зарплата (есть есть отдельный блок в карточке вакансии и зарплату можно вытащить из тега)
- количество кандидатов (если есть)
- дата публикации вакансии 
- сфера деятельности компании (если есть)
- текст вакансии

### Импорт библиотек

In [ ]:
!pip install backoff 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd
import numpy as np

import json
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import os

import logging
import backoff

### Делаем выгрузку из HeadHunter
Ссылка на API HH: https://github.com/hhru/api \
Для того, чтобы написать запрос используется язык поисковых запросов, задокументированный тут: https://hh.ru/article/1175 \
Фильтрацию можно делать в самом запросе, например, сделать поиск по полю компаний или же по наличию конкретного текста в описании, где COMPANY_NAME - название компании, а DESCRIPTION - описание вакансии. Язык запросов работает и на обычном сайте hh.ru, что бывает полезно.

Создадим необходимые константы: BASE_URL

In [ ]:
BASE_URL = "https://api.hh.ru/"

TEXT = (f'SQL NAME:("Аналитик" or "Analyst" or "Data Scientist"'
       'or "Дата саентист" or "ML")')

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=1)
DATE_TO = dt_now.isoformat() #API требует дату в формате ISO 8601 YYYY-MM-DD 
DATE_FROM = dt_from.isoformat()

VAC_URL = (BASE_URL +
          f'vacancies?text={TEXT}&date_from={DATE_FROM}&date_to={DATE_TO}'
          '&responses_count_enabled=True&per_page=100')

In [ ]:
VAC_URL

'https://api.hh.ru/vacancies?text=SQL NAME:("Аналитик" or "Analyst" or "Data Scientist"or "Дата саентист" or "ML")&date_from=2023-04-16&date_to=2023-04-17&responses_count_enabled=True&per_page=100'

In [ ]:
vacancy_df = pd.DataFrame() # создаем датафрейм в который будем сохранять наши данные

В выдаче API HH нет опыта работы, поэтому мы укажем его в запросе самостоятельно.Нам нужны вокансии уровня junior. Оставим вокансии без опыта и с опытом 1-3 года 

In [ ]:
# Определение функции-обработчика ошибок
@backoff.on_exception(backoff.expo, Exception, max_time=10)
def send_get_request(url):
    r = requests.get(url)
    r.raise_for_status()
    return r

# Настройка уровня логирования (можно указать другой уровень)
logging.basicConfig(level=logging.DEBUG)

# Определение URL-строки с GET-параметрами
experiences = ['noExperience', 'between1And3']
for exp in experiences:
    page = 0
    url = VAC_URL + f'&experience={exp}'
    try:
        # Отправка GET-запроса и получение данных с использованием backoff
        r = send_get_request(url + f'&page={page}')
        data = json.loads(r.text)
        logging.debug(f'{exp} - {data["pages"]}')
        items = data['items']

        # Обработка всех страниц данных
        for page in range(1, data['pages']):
            r = send_get_request(url + f'&page={page}')
            data = json.loads(r.text)
            items += data['items']

        # Создание DataFrame и добавление столбца с опытом работы
        df = pd.DataFrame(items)
        df['experience'] = exp

        # Объединение с основным DataFrame
        vacancy_df = pd.concat([vacancy_df, df])
    except Exception as e:
        # Логирование ошибки
        logging.error(f'{exp} - {e}')

In [ ]:
vacancy_df.reset_index(drop=True, inplace=True)

### Вытащим ценную информацию из словарей

In [ ]:
def return_id(x, key='id', nan_value=np.nan):
    if x is not None:
        return x.get(key, nan_value)
    else:
        return None

In [ ]:
vacancy_df['company']= vacancy_df['employer'].apply(return_id, key= 'name')
vacancy_df['company_field']= vacancy_df['department'].apply(return_id, key= 'name')
vacancy_df['salary_from']= vacancy_df['salary'].apply(return_id, key= 'from')\
.astype(str)
vacancy_df['salary_to']= vacancy_df['salary'].apply(return_id, key= 'to')\
.astype(str)
vacancy_df['currency']= vacancy_df['salary'].apply(return_id, key= 'currency')
vacancy_df['salaries'] = (vacancy_df['salary_from'] + '-' +
                         vacancy_df['salary_to'] + ' ' +
                         vacancy_df['currency'])
vacancy_df['location']= vacancy_df['area'].apply(return_id, key= 'name')
vacancy_df['job_type']= vacancy_df['employment'].apply(return_id, key= 'name')
vacancy_df['skills']= vacancy_df['snippet'].apply(return_id, key= 'requirement')
vacancy_df['date']=pd.to_datetime(vacancy_df['published_at']).dt.date

In [ ]:
# ошибки возникают исключительно если парсинг запускался несколько раз в день - 
#hh просто блокирует запросы, использование Selenium дает тот же самый результат
list_exp = []
def get_description(cell):
    try:
        return BeautifulSoup(json.loads(requests.get(cell).text)['description'])\
        .get_text() 
    except:
        list_exp.append(f'{cell} doesnt work')
        return None

vacancy_df['description']= vacancy_df['url'].apply(get_description)
list_exp

[]

In [ ]:
#удалим лишний столбец
vacancy_df.drop('salary', axis=1, inplace=True)

# Переименование столбцов
vacancy_df.rename(columns={'name': 'title',
                   'alternate_url': 'link',
                   'salaries': 'salary'}, inplace=True)
# Создание столбцов
vacancy_df.loc[:, 'country'] = 'Россия'
vacancy_df.loc[:, 'source'] = 'hh'
vacancy_df.loc[:, 'skills'] = None

### Посмотрим содержание вакансий

In [ ]:
# Удаляем лишнее
df = vacancy_df[['title',  'company', 'country', 'location', 'salary', 'source',
                 'link', 'date', 'company_field', 'description', 'skills',
                 'job_type']]

In [ ]:
df

,title,company,country,location,salary,source,link,date,company_field,description,skills,job_type
0,Аналитик,SPAR Калининград,Россия,Калининград,69000.0-nan RUR,hh,https://hh.ru/vacancy/78754789,2023-04-17,None,ТС «SPAR-Калининград» ищет в свою команду колл...,None,Полная занятость
1,Аналитик,Группа компаний С7,Россия,Новосибирск,NaN,hh,https://hh.ru/vacancy/67116979,2023-04-17,None,ЗАО «Группа компаний С7» (S7 Group) — управляю...,None,Полная занятость
2,Аналитик,Группа компаний С7,Россия,Москва,NaN,hh,https://hh.ru/vacancy/67116978,2023-04-17,None,ЗАО «Группа компаний С7» (S7 Group) — управляю...,None,Полная занятость
3,Аналитик со знанием SQL,ЛОКО-БАНК,Россия,Самара,NaN,hh,https://hh.ru/vacancy/79388365,2023-04-17,None,Мы ждем от соискателя: Опыт написания SQL-зап...,None,Полная занятость
4,Аналитик данных,BRANDPOL,Россия,Екатеринбург,40000.0-80000.0 RUR,hh,https://hh.ru/vacancy/78721623,2023-04-17,None,Мы международная команда единомышленников: юри...,None,Полная занятость
...,...,...,...,...,...,...,...,...,...,...,...,...
319,Аналитик данных Управления анализа и мониторин...,АЛЬФА-БАНК,Россия,Минск,NaN,hh,https://hh.ru/vacancy/78665018,2023-04-16,None,Приглашаем в команду Управления анализа и мони...,None,Полная занятость
320,Аналитик данных,Teamline Consult,Россия,Москва,150000.0-nan RUR,hh,https://hh.ru/vacancy/78691511,2023-04-16,None,Цель Сбера сейчас — это создание аналитических...,None,Полная занятость
321,Аналитик (Terrasoft Creatio CRM),АКБ «ЦентроКредит»,Россия,Москва,NaN,hh,https://hh.ru/vacancy/77498276,2023-04-17,None,Банку требуется специалист отдела информационн...,None,Полная занятость
322,Аналитик по информационной безопасности,МКБ Инвестиции,Россия,Москва,NaN,hh,https://hh.ru/vacancy/78686608,2023-04-17,None,Обязанности: Анализ информационных систем на ...,None,Полная занятость


In [ ]:
from google.colab import drive

Mounted at /content/drive


In [ ]:
from datetime import date
from google.colab import drive
drive.mount("/content/drive")
df.to_csv (f'/content/drive/MyDrive/hh_{date.today()}.csv', index = None, 
           header=True,  encoding='utf-8')

Файл успешно сохранен как: /content/drive/MyDrive/hh_2023-04-17.csv
